In [7]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import cm
%matplotlib inline
from matplotlib import pyplot as plt
from statsmodels.distributions.empirical_distribution import ECDF
import datetime
from matplotlib.colors import TwoSlopeNorm
import json
import geopandas as gpd
import requests
import plotly.express as px
from shapely import wkt
import os
import geojson
import csv

from natsort import natsorted
from PIL import Image

import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point

#Avoiding Type 3 fonts in matplotlib plots
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [8]:
font = {'size'   : 20}

matplotlib.rc('font', **font)
matplotlib.rc('lines', linewidth=2.0)
matplotlib.rc('lines', markersize=8)

In [9]:
pd.set_option('display.max_rows', 500)

In [10]:
res='results_covid/'

## RKI Data reading

In [11]:
# reading and drop the column LKNR
df1=pd.read_csv('data/7-days incidences1.csv').drop(['LKNR'], axis=1)

df2=pd.read_csv('data/7-days incidences2.csv').drop(['LKNR'], axis=1)

In [15]:
df1.head()

,LK,18.11.2020,19.11.2020,20.11.2020,21.11.2020,22.11.2020,23.11.2020,24.11.2020,25.11.2020,26.11.2020,...,2021/9/1,2021/9/2,2021/9/3,2021/9/4,2021/9/5,2021/9/6,2021/9/7,2021/9/8,2021/9/9,2021/9/10
0,LK Ahrweiler,143.8,156.1,159.1,156.1,172.2,139.1,139.9,136.8,119.9,...,122.7,148.8,188.0,177.0,199.3,183.4,197.2,208.7,171.8,191.8
1,LK Aichach-Friedberg,144.8,138.9,158.2,173.8,167.8,156.0,139.6,121.8,95.1,...,68.9,90.4,98.6,102.2,99.2,92.7,92.7,94.9,68.2,71.2
2,LK Alb-Donau-Kreis,114.7,117.2,123.8,124.8,121.8,122.8,118.2,123.3,112.6,...,70.3,88.5,95.5,107.0,89.3,104.6,103.1,91.0,84.4,78.9
3,LK Altenburger Land,145.4,153.3,129.8,205.8,212.5,200.2,219.3,236.0,252.8,...,21.4,22.5,30.4,28.3,27.2,27.1,27.1,44.0,40.6,36.1
4,LK Altenkirchen,63.7,58.2,66.8,90.1,97.8,104.8,104.8,102.5,87.7,...,91.5,103.9,106.2,113.9,104.6,103.1,117.1,114.0,98.5,98.5


In [ ]:
df=df1.set_index('LK').join(df2.set_index('LK'))
df.columns

In [ ]:
#covert to date time, specify the format
df.columns = pd.to_datetime(df.columns,infer_datetime_format=True,dayfirst=True).strftime("%Y-%m-%d")
# sort the columns by date
df = df.reindex(sorted(df.columns), axis=1)
df

In [ ]:
# The average 7-days incidences over per week
Avg_7days_incidences=df.groupby(np.arange(len(df.columns)) // 7, axis=1).mean()
# rename the columns
# Avg_7days_incidences.columns=df.columns[::7]
#store
Avg_7days_incidences.to_csv(res+'average 7-days incidences.csv')
# Avg_7days_incidences

## Create Map

In [ ]:
# Reading files
df_7days_incidences=pd.read_csv(res+'average 7-days incidences.csv')

gis = gpd.read_file('data/sehr_hoch.geo.json')
# gpd.datasets.available
gis.crs

In [ ]:
'''
Mapping part of gis info
'''
df=pd.DataFrame()
#iterrating each row in gis data
for idx, row in gis.iterrows():
        #mapping the name of landkreis in Germany
        temp=df_7days_incidences[df_7days_incidences['LK'].str.contains(gis['NAME_3'][idx])]
        #only find the exact one
        if len(temp)==1:
            #index in df_7days_incidences 
            new_idx=temp.index
            # find the mapping row
            df1=df_7days_incidences.iloc[temp.index]
            df2=pd.DataFrame(row).T
            #concat in columns
            df_new=pd.concat([df1.reset_index(drop=True),df2.reset_index(drop=True)],axis=1)
            df_new=df_new.set_index(new_idx)
            
            df=pd.concat([df,df_new])  
        else:
            pass

In [ ]:
# df.sort_index().to_csv(res+'part of mapped 7days_incidences with geo info.csv')
# df.sort_index().to_excel(res+'part of mapped 7days_incidences with geo info.xlsx')
# df_7days_incidences.to_excel(res+'average 7-days incidences.xlsx')
gis.to_csv(res+'gis info.csv')
gis.to_excel(res+'gis info.xlsx')

In [ ]:
##reading the file which mapped manually
gis_incidences=pd.read_excel(res+'mapped 7days_incidences with geo info.xlsx',index_col=[0])
gis_incidences.shape

In [ ]:
#take the rows where geometry is not NA:
gis_incidences_nona = gis_incidences[gis_incidences['geometry'].notna()]
gis_incidences_nona.shape

In [ ]:
# IMPORTANT!!!
gis_incidences_nona=gis_incidences_nona.reset_index(drop=True)

In [ ]:
#multipolygon
'''
Aurich  14
Bergstraße 24
Cuxhaven 42
Dithmarschen 50
Friesland 71
Karlsruhe 115
Konstanz 120
Lindau 132
Nordfriesland 173
Nordwestmecklenburg 177
Oberallgäu 180
Ostholstein 194
Pinneberg 200
Rendsburg-Eckernförde 210
Rostock 223  Rostock Städte
Schleswig-Flensburg 238
Stadtverband Saarbrücken 253  Saarbrücken Städte
Vorpommern-Rügen 279 Rügen
Wesermarsch 290
Wittmund 294
Wolfenbüttel 295
SK Landau i.d.Pfalz 358 Landau Städte
SK Rostock 386 Rostock Städte
'''

In [ ]:
'''
To complete the multipolygon records since the limited storage in excel file, 
the multipolygon records which stored in the excel file is incomplete
'''
def check_multipolygon(land,df_incidence,df_gis,covid_re,gis_re):
    if df_incidence.iloc[df_incidence.index[df_incidence[land]==covid_re].values[0],116].startswith('MULTI'):
        df_incidence.iloc[df_incidence.index[df_incidence[land]==covid_re].values[0],116]=df_gis.geometry[df_gis.index[df_gis['NAME_3']==gis_re].values[0]]
    else:
        pass
        

In [ ]:
#MULTIPOLYGON (which doesn't contain all data in the cell in excel file)

check_multipolygon('LK',gis_incidences_nona,gis,'LK Aurich','Aurich')
check_multipolygon('LK',gis_incidences_nona,gis,'LK Bergstraße','Bergstraße')
check_multipolygon('LK',gis_incidences_nona,gis,'LK Cuxhaven','Cuxhaven')
check_multipolygon('LK',gis_incidences_nona,gis,'LK Dithmarschen','Dithmarschen')
check_multipolygon('LK',gis_incidences_nona,gis,'LK Friesland','Friesland')
check_multipolygon('LK',gis_incidences_nona,gis,'LK Karlsruhe','Karlsruhe')
check_multipolygon('LK',gis_incidences_nona,gis,'LK Konstanz','Konstanz')
check_multipolygon('LK',gis_incidences_nona,gis,'LK Lindau','Lindau')
check_multipolygon('LK',gis_incidences_nona,gis,'LK Nordfriesland','Nordfriesland')
check_multipolygon('LK',gis_incidences_nona,gis,'LK Nordwestmecklenburg','Nordwestmecklenburg')
check_multipolygon('LK',gis_incidences_nona,gis,'LK Oberallgäu','Oberallgäu')
check_multipolygon('LK',gis_incidences_nona,gis,'LK Ostholstein','Ostholstein')
check_multipolygon('LK',gis_incidences_nona,gis,'LK Pinneberg','Pinneberg')
check_multipolygon('LK',gis_incidences_nona,gis,'LK Rendsburg-Eckernförde','Rendsburg-Eckernförde')
check_multipolygon('LK',gis_incidences_nona,gis,'LK Rostock','Bad Doberan')
check_multipolygon('LK',gis_incidences_nona,gis,'LK Schleswig-Flensburg','Schleswig-Flensburg')
check_multipolygon('LK',gis_incidences_nona,gis,'LK Stadtverband Saarbrücken','Saarbrücken Städte')
check_multipolygon('LK',gis_incidences_nona,gis,'LK Vorpommern-Rügen','Rügen')
check_multipolygon('LK',gis_incidences_nona,gis,'LK Wesermarsch','Wesermarsch')
check_multipolygon('LK',gis_incidences_nona,gis,'LK Wittmund','Wittmund')
check_multipolygon('LK',gis_incidences_nona,gis,'LK Wolfenbüttel','Wolfenbüttel')
check_multipolygon('LK',gis_incidences_nona,gis,'SK Landau i.d.Pfalz','Landau Städte')
check_multipolygon('LK',gis_incidences_nona,gis,'SK Rostock','Rostock Städte')
check_multipolygon('LK',gis_incidences_nona,gis,'LK Vorpommern-Rügen 1','Nordvorpommern')
check_multipolygon('LK',gis_incidences_nona,gis,'LK Vorpommern-Greifswald','Ostvorpommern')
check_multipolygon('LK',gis_incidences_nona,gis,'LK Vorpommern-Greifswald 1','Uecker-Randow')

#POLYGON
gis_incidences_nona.iloc[gis_incidences_nona.index[gis_incidences_nona['LK']=='LK Waldshut'].values[0],116]=gis.geometry[gis.index[gis['NAME_3']=='Waldshut'].values[0]]
gis_incidences_nona.iloc[gis_incidences_nona.index[gis_incidences_nona['LK']=='SK Wilhelmshaven'].values[0],116]=gis.geometry[gis.index[gis['NAME_3']=='Wilhelmshaven Städte'].values[0]]
gis_incidences_nona.iloc[gis_incidences_nona.index[gis_incidences_nona['LK']=='SK Kiel'].values[0],116]=gis.geometry[gis.index[gis['NAME_3']=='Kiel Städte'].values[0]]
gis_incidences_nona.iloc[gis_incidences_nona.index[gis_incidences_nona['LK']=='LK Plön'].values[0],116]=gis.geometry[gis.index[gis['NAME_3']=='Plön'].values[0]]


In [ ]:
# gis_incidences_nona['geometry'][210]
type(gis['geometry'][gis['NAME_3']=='Landau Städte'])
# gis[gis['NAME_3']=='Cuxhaven']['geometry']
# gpd.GeoSeries.from_wkt(gis_incidences_nona['geometry'][12])

In [ ]:
# for i in gis_incidences_nona['geometry']:
for idx, content in gis_incidences_nona['geometry'].iteritems():
    if idx>403:
        if type(content) is str:
            print(idx)
            gis_incidences_nona['geometry'][idx]=wkt.loads(content)

In [ ]:
#Converting a column of Polygons from string to GeoPandas geometry
# gis_incidences_nona.head['geometry'] = gpd.GeoSeries.from_wkt(gis_incidences_nona.head['geometry'])

# gis_incidences_nona.head(10)['geometry'] = gis_incidences_nona.head(10)['geometry'].apply(lambda x: wkt.loads(x))

# gis_incidences_nona.head(10)['geometry'] = gis_incidences_nona.head(10)['geometry'].apply(wkt.loads)

In [ ]:
# Convert to Geo data
gis_incidences_nona = gpd.GeoDataFrame(
    gis_incidences_nona,crs="EPSG:4326", geometry=gis_incidences_nona.geometry)
gis_incidences_nona

In [ ]:
gis_incidences_nona.to_pickle(res+'gis_incidences_nona.pkl')

In [ ]:
# Get the rows of df sorted by the n largest values of columns
gis_incidences_nona.nlargest(10, '29').iloc[:,[0,30,-6,-8,-10]]

## Maps generation and Gif Map

In [ ]:
"""
Read geo data
"""
gis_incidences_nona=pd.read_pickle(res+'gis_incidences_nona.pkl')
gis_incidences_nona

In [ ]:
%%time
''' Get new gis_incidences_nona
 4 weeks(points) in a row to start colouring from the second one
 check the ratio of each two adjacents is greater than THRESHOLD
 keep all True value, inplace false with 0
'''
sub_gis_incidences=gis_incidences_nona.iloc[:,1:103]
sub_gis_incidences=sub_gis_incidences.reset_index(drop=True)

# check the ratio of each two adjacents is greater than SHRESHOLD
THRESHOLD = 1.2
# 4 points in a row start to colouring
for idx, row in sub_gis_incidences.iterrows():

    #initial the color each row(Landkreis)
    c_line=pd.Series([False]*sub_gis_incidences.shape[1])
    #incidences numebr for each landkreis along all weeks
    val=sub_gis_incidences.loc[idx,:]
    for i in range(1,len(row)-2):
        if (row[i]>THRESHOLD*row[i-1])&(row[i+1]>THRESHOLD*row[i])&(row[i+2]>THRESHOLD*row[i+1]):
#             c_line[i-1]=True
            c_line[i]=True
            c_line[i+1]=True
            c_line[i+2]=True
    #keep all value according true, inplace false with 0
#     print(len(c_line),len(val))
    val=val.where(c_line.values,0)
    sub_gis_incidences.loc[idx,:]=val

#assign values back to gis_incidences    
gis_incidences_nona.iloc[:,1:103]=sub_gis_incidences

In [ ]:
%%time

'''
4 weeks in a row but start coloring from the second one
check the ratio of each two adjacents is greater than THRESHOLD
keep all True value, inplace false with 0
'''
# save all the maps in the charts folder
output_path = res+'maps_4consecutiveWeeks_Oct2023'

# list of weeks (which are the column names at the moment)
list_of_weeks = gis_incidences_nona.iloc[:,1:103].columns.tolist()

# set the min and max range for the choropleth map
# vmin, vmax = 0, 1200

# create an annotation for the year by grabbing the first 4 digits

# start the for loop to create one map per year
for week in list_of_weeks:
    
#     fig, ax = plt.subplots(1, 1,figsize=(20, 10))
    norm1 = TwoSlopeNorm(vmin=0, vcenter=0.5, vmax=16) 
    # create map, UDPATE: added plt.Normalize to keep the legend range the same for all maps
    fig = gis_incidences_nona.plot(column=week, cmap='Reds', figsize=(10,10), linewidth=0.8, edgecolor='0.8',
legend=False, norm=norm1)
    
    # remove axis of chart
    fig.axis('off')
    
    # add a title
#     fig.set_title('Average 7 day incidence rate in Germany', fontdict={'fontsize': '20','fontweight' : '3'})
    # Add prefix of column labels
    only_week = 'Week '+week
    # position the annotation to the bottom left
    fig.annotate(only_week,
            xy=(0.1, .18), xycoords='figure fraction',
            horizontalalignment='left', verticalalignment='top',
            fontsize=20)
    
    # this will save the figure as a high-res png in the output path. you can also save as svg if you prefer.
    filepath = os.path.join(output_path, only_week+'.pdf')
    filepath = os.path.join(output_path, only_week+'.png')
    chart = fig.get_figure()
    chart.savefig(filepath, dpi=100,bbox_inches='tight')
    plt.close(chart)

In [ ]:
%%time
'''
For overleaf ----4 weeks in a row
'''
# save all the maps in the charts folder
output_path = res+'maps_overleaf'

# list of weeks (which are the column names at the moment)
list_of_weeks = gis_incidences_nona.iloc[:,1:103].columns.tolist()

# set the min and max range for the choropleth map
# vmin, vmax = 0, 1200

# create an annotation for the year by grabbing the first 4 digits

# start the for loop to create one map per year
for week in list_of_weeks:
    if int(week)>30 and int(week)<35:
        norm1 = TwoSlopeNorm(vmin=0, vcenter=0.5, vmax=16) 
        # create map, UDPATE: added plt.Normalize to keep the legend range the same for all maps
        fig = gis_incidences_nona.plot(column=week, cmap='Reds', figsize=(10,10), linewidth=0.9, rasterized=False, edgecolor='0.1',
    legend=False, norm=norm1)

        # remove axis of chart
        fig.axis('off')

        # add a title
#         fig.set_title('Average 7 day incidence rate in Germany', fontdict={'fontsize': '20','fontweight' : '3'})
        # Add prefix of column labels
        if int(week)==31:
            only_week ='Week 1'
        elif int(week)==32:
            only_week='Week 2'
        elif int(week)==33:
            only_week='Week 3'
        elif int(week)==34:
            only_week='Week 4'
        # position the annotation to the bottom left
        fig.annotate(only_week,
                xy=(0.1, .165), xycoords='figure fraction',
                horizontalalignment='left', verticalalignment='top',
                fontsize=20)

        # this will save the figure as a high-res png in the output path. you can also save as svg if you prefer.
#         filepath = os.path.join(output_path, only_week+'.pdf')
        filepath = os.path.join(output_path, only_week+'.pdf')
        chart = fig.get_figure()
        chart.savefig(filepath, dpi=100, bbox_inches='tight')
#     plt.close(chart)